# Step-A 初步训练

### 1.加载VGG预训练参数
### 2.冻结CONV层，训练FC层
### 3.保存FC层参数

In [1]:
import os
import random
import tensorflow as tf 
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from data_preprocess import *
from image_preprocess import *
from bcnn_model import *
from image_enhancement import *

In [4]:
weights_file = "vgg16_weights.npz"
weights_file_last_layer = "bcnn_last_weights.npz"
weights_file_conv = "bcnn_conv_weights.npz"

batch_size = 32
lr = 0.001
finetune_step = -1

image_size = 224
n_classes = 12

epoch = 50

In [5]:
tf.reset_default_graph()
sess = tf.Session()
# build input
imgs = tf.placeholder(tf.float32, shape = [None, image_size, image_size, 3], name = "input_x")
targets = tf.placeholder(tf.int32, shape = [None, n_classes], name = "input_y")
keep_prob = tf.placeholder(tf.float32, name = "keep_prob")

# build cal graph
bilinear_cnn = vgg_bilinear_model(imgs, 
                                  keep_prob, 
                                  weights_file = weights_file, 
                                  weights_file_last = weights_file_last_layer, 
                                  sess = sess, 
                                  finetune = False)

# build cal node parameter
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = targets, logits = bilinear_cnn.fc3l)
cost = tf.reduce_mean(cross_entropy, name = "cost")
optimizer = tf.train.MomentumOptimizer(learning_rate=0.2, momentum=0.5).minimize(cost)

predicted = tf.nn.softmax(bilinear_cnn.fc3l, name = "predicted")
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name = "accuracy")

# start train
sess.run(tf.global_variables_initializer())
bilinear_cnn.load_vgg_weights(sess)

for v in tf.trainable_variables():
    print("Trainable variables", v)
print('Starting training')


###################################
train_img_batch, train_label_batch = get_batchs(tfrecord_filename = "train.tfrecords", 
                                                    image_size = image_size, classes = n_classes,
                                                    batch_size = batch_size, min_after_dequeue = 500,
                                                   use_data_enhancement = True)
test_img_batch, test_label_batch = get_batchs(tfrecord_filename = "test.tfrecords", 
                                                  image_size = image_size, classes = n_classes,
                                                  batch_size = batch_size, min_after_dequeue = 500,
                                                     use_data_enhancement = False)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for i in range(int((4512 / batch_size) * epoch)):
#for i in range(2):
    # get data
    train_imgs, train_labels= sess.run([train_img_batch, train_label_batch])
    test_imgs, test_labels= sess.run([test_img_batch, test_label_batch])

#    not random rotate
#    train_imgs = image_random_enhancement(train_imgs)

    # train single loop
    _, train_loss = sess.run([optimizer,cost], feed_dict={imgs: train_imgs, targets: train_labels, keep_prob: 0.5})
    #train_loss = sess.run(cost, feed_dict={imgs: train_imgs, targets: train_labels, keep_prob: 1.})
    print("train loss: %10f"%(train_loss))
            
    # print test stat
    if (i % 50 == 0):
        print("########### epoch %d ###########"%((i / (4512 / batch_size)) + 1))
        print("########### loop %d ###########"%(i))
        test_loss, test_acc = sess.run([cost, accuracy], feed_dict={imgs: test_imgs, targets: test_labels, keep_prob: 1.})
        print("test loss: %10f   test accuracy: %10f"%(test_loss, test_acc))
        print("########### loop %d ###########"%(i))

# close data queue
coord.request_stop()
coord.join(threads)

# save first train parameters
last_layer_weights = []

for var in bilinear_cnn.last_layer_parameters:
    last_layer_weights.append(sess.run(var))
print(last_layer_weights)

np.savez(weights_file_last_layer, last_layer = last_layer_weights)

# end
sess.close()

Adding Data Augmentation
Shape of z (?, 262144)
Adding weights to conv1_1/b:0
Adding weights to conv1_1/W:0
Adding weights to conv1_2/b:0
Adding weights to conv1_2/W:0
Adding weights to conv2_1/b:0
Adding weights to conv2_1/W:0
Adding weights to conv2_2/b:0
Adding weights to conv2_2/W:0
Adding weights to conv3_1/b:0
Adding weights to conv3_1/W:0
Adding weights to conv3_2/b:0
Adding weights to conv3_2/W:0
Adding weights to conv3_3/b:0
Adding weights to conv3_3/W:0
Adding weights to conv4_1/b:0
Adding weights to conv4_1/W:0
Adding weights to conv4_2/b:0
Adding weights to conv4_2/W:0
Adding weights to conv4_3/b:0
Adding weights to conv4_3/W:0
Adding weights to conv5_1/b:0
Adding weights to conv5_1/W:0
Adding weights to conv5_2/b:0
Adding weights to conv5_2/W:0
Adding weights to conv5_3/b:0
Adding weights to conv5_3/W:0
Trainable variables <tf.Variable 'fc-new/W:0' shape=(262144, 12) dtype=float32_ref>
Trainable variables <tf.Variable 'fc-new/b:0' shape=(12,) dtype=float32_ref>
Starting tr

train loss:   2.226629
train loss:   2.126839
train loss:   2.119179
train loss:   1.984917
train loss:   2.217154
train loss:   2.312849
train loss:   2.182417
train loss:   2.133904
train loss:   2.022936
train loss:   2.066675
train loss:   2.020809
train loss:   2.147822
train loss:   2.137505
train loss:   2.230508
train loss:   2.080014
train loss:   2.302559
train loss:   2.161315
train loss:   2.285050
train loss:   2.269837
train loss:   2.324056
train loss:   2.033614
train loss:   1.932268
train loss:   2.140348
train loss:   2.180972
train loss:   2.031968
train loss:   2.277529
########### epoch 3 ###########
########### loop 300 ###########
test loss:   2.017839   test accuracy:   0.625000
########### loop 300 ###########
train loss:   2.171234
train loss:   2.103474
train loss:   1.932008
train loss:   1.903594
train loss:   2.082652
train loss:   2.091609
train loss:   2.122979
train loss:   1.956491
train loss:   2.065399
train loss:   2.249638
train loss:   2.221706
t

train loss:   1.936234
train loss:   1.971342
train loss:   2.058169
train loss:   1.966018
train loss:   2.166979
train loss:   2.054907
train loss:   1.908723
train loss:   1.994505
########### epoch 5 ###########
########### loop 600 ###########
test loss:   1.888732   test accuracy:   0.531250
########### loop 600 ###########
train loss:   1.868687
train loss:   2.073086
train loss:   1.972253
train loss:   1.920221
train loss:   2.114712
train loss:   2.051690
train loss:   1.987807
train loss:   1.948579
train loss:   1.994817
train loss:   1.984885
train loss:   1.890932
train loss:   2.064524
train loss:   1.764801
train loss:   1.849709
train loss:   1.731882
train loss:   2.056951
train loss:   2.068863
train loss:   1.973662
train loss:   1.760360
train loss:   1.901813
train loss:   2.121765
train loss:   1.944566
train loss:   1.720959
train loss:   1.983501
train loss:   2.082654
train loss:   2.020010
train loss:   2.126195
train loss:   1.909852
train loss:   1.963236
t

train loss:   1.960027
train loss:   2.023679
train loss:   1.986275
train loss:   1.923365
train loss:   1.964178
train loss:   2.042396
train loss:   1.897279
train loss:   1.782140
train loss:   1.788378
train loss:   1.730770
train loss:   1.839459
train loss:   1.882546
train loss:   1.917056
train loss:   2.126499
train loss:   1.782447
train loss:   1.736281
train loss:   1.824094
train loss:   1.791484
train loss:   1.770582
train loss:   1.892394
train loss:   1.910802
train loss:   2.051530
train loss:   1.771577
train loss:   1.990059
train loss:   1.978802
train loss:   1.770191
train loss:   1.790993
train loss:   2.038875
train loss:   1.817622
train loss:   2.053255
train loss:   1.703997
train loss:   1.759703
train loss:   1.926943
train loss:   1.761991
train loss:   1.722384
train loss:   1.888822
train loss:   2.088958
train loss:   1.684421
train loss:   1.661585
train loss:   2.121170
train loss:   1.688696
train loss:   1.848740
train loss:   1.448784
train loss:

train loss:   1.557945
train loss:   1.803572
train loss:   1.642777
train loss:   1.925966
train loss:   1.929476
train loss:   1.968287
train loss:   1.768882
train loss:   1.953325
train loss:   1.931266
train loss:   1.872196
train loss:   1.704076
train loss:   1.653341
train loss:   1.717947
train loss:   1.782638
train loss:   1.886397
train loss:   1.834176
train loss:   1.695913
train loss:   1.808748
train loss:   1.713383
train loss:   1.831076
train loss:   1.678157
train loss:   1.757102
train loss:   1.525454
train loss:   1.669239
train loss:   1.649759
train loss:   1.738259
train loss:   1.958512
train loss:   1.578258
########### epoch 9 ###########
########### loop 1250 ###########
test loss:   1.596338   test accuracy:   0.718750
########### loop 1250 ###########
train loss:   1.893358
train loss:   1.739036
train loss:   1.712085
train loss:   1.713762
train loss:   1.894717
train loss:   1.679202
train loss:   1.961109
train loss:   2.023680
train loss:   1.482583

...

train loss:   1.610539
train loss:   1.506005
train loss:   1.674107
train loss:   1.452937
train loss:   1.540440
train loss:   1.467504
train loss:   1.701408
train loss:   1.601833
train loss:   1.439782
train loss:   1.596969
train loss:   1.399081
train loss:   1.348963
########### epoch 41 ###########
########### loop 5650 ###########
test loss:   1.090608   test accuracy:   0.812500
########### loop 5650 ###########
train loss:   1.461149
train loss:   1.402635
train loss:   1.271405
train loss:   1.530966
train loss:   1.645345
train loss:   1.561751
train loss:   1.601410
train loss:   1.580663
train loss:   1.563493
train loss:   1.661891
train loss:   1.638674
train loss:   1.833112
train loss:   1.651967
train loss:   1.540776
train loss:   1.391548
train loss:   1.304595
train loss:   1.661882
train loss:   1.910814
train loss:   1.540810
train loss:   1.329898
train loss:   1.734295
train loss:   1.486135
train loss:   1.329811
train loss:   1.702499
train loss:   1.63439

train loss:   1.364202
train loss:   1.477433
train loss:   1.594065
train loss:   1.543890
train loss:   1.566730
train loss:   1.333349
train loss:   1.366247
train loss:   1.625342
train loss:   1.302697
train loss:   1.580124
train loss:   1.529928
train loss:   1.724994
train loss:   1.444337
train loss:   1.508520
train loss:   1.663736
train loss:   1.870498
train loss:   1.309711
train loss:   1.495309
train loss:   1.690284
train loss:   1.522724
train loss:   1.419843
train loss:   1.523471
train loss:   1.343464
train loss:   1.568757
train loss:   1.353173
train loss:   1.832630
train loss:   1.402715
train loss:   1.187060
train loss:   1.546044
train loss:   1.499399
train loss:   1.553142
train loss:   1.363056
train loss:   1.468595
train loss:   1.562726
train loss:   1.416784
train loss:   1.515698
train loss:   1.511946
train loss:   1.392161
train loss:   1.488318
train loss:   1.325569
train loss:   1.098652
train loss:   1.590992
train loss:   1.468733
train loss:

train loss:   1.461969
train loss:   1.401904
train loss:   1.327950
train loss:   1.405113
train loss:   1.375396
train loss:   1.488687
train loss:   1.315737
train loss:   1.685784
train loss:   1.683614
train loss:   1.350252
train loss:   1.714650
train loss:   1.418190
train loss:   1.424150
train loss:   1.563927
train loss:   1.333528
train loss:   1.532577
train loss:   1.537470
train loss:   1.555873
train loss:   1.528166
train loss:   1.560899
train loss:   1.211607
train loss:   1.578369
train loss:   1.445204
train loss:   1.172449
train loss:   1.398249
train loss:   1.338835
train loss:   1.223405
train loss:   1.712476
train loss:   1.359718
train loss:   1.638365
train loss:   1.307589
train loss:   1.088282
train loss:   1.360795
########### epoch 45 ###########
########### loop 6300 ###########
test loss:   1.045294   test accuracy:   0.812500
########### loop 6300 ###########
train loss:   1.523690
train loss:   1.388853
train loss:   1.537597
train loss:   1.47805

train loss:   1.622998
train loss:   1.533355
train loss:   1.263442
train loss:   1.528119
train loss:   1.589951
train loss:   1.371045
train loss:   1.184609
train loss:   1.686881
train loss:   1.338908
train loss:   1.413891
train loss:   1.603187
train loss:   1.470304
train loss:   1.233437
train loss:   1.280746
train loss:   1.186832
train loss:   1.213359
########### epoch 47 ###########
########### loop 6600 ###########
test loss:   0.998325   test accuracy:   0.750000
########### loop 6600 ###########
train loss:   1.574151
train loss:   1.502000
train loss:   1.508882
train loss:   1.306795
train loss:   1.544613
train loss:   1.286957
train loss:   1.237090
train loss:   1.426738
train loss:   1.703570
train loss:   1.569356
train loss:   1.261774
train loss:   1.222030
train loss:   1.694701
train loss:   1.251956
train loss:   1.666167
train loss:   1.245322
train loss:   1.610581
train loss:   1.379456
train loss:   1.414232
train loss:   1.539229
train loss:   1.46403

test loss:   0.894696   test accuracy:   0.875000
########### loop 6900 ###########
train loss:   1.617668
train loss:   1.467458
train loss:   1.255551
train loss:   1.229854
train loss:   1.045715
train loss:   1.577470
train loss:   1.459185
train loss:   1.243940
train loss:   1.372841
train loss:   1.356395
train loss:   1.543184
train loss:   1.417125
train loss:   1.697636
train loss:   1.371930
train loss:   1.283865
train loss:   1.149129
train loss:   1.426818
train loss:   1.569421
train loss:   1.693427
train loss:   1.370137
train loss:   1.378608
train loss:   1.621001
train loss:   1.526272
train loss:   1.257567
train loss:   1.616010
train loss:   1.364344
train loss:   1.621508
train loss:   1.328825
train loss:   1.291275
train loss:   1.611421
train loss:   1.402042
train loss:   1.413630
train loss:   1.400083
train loss:   1.272099
train loss:   1.682045
train loss:   1.547058
train loss:   1.444280
train loss:   1.650397
train loss:   1.498969
train loss:   1.392

In [ ]:
weights_dict_test = np.load(weights_file_last_layer, encoding = 'bytes')

In [4]:
print(weights_dict_test.keys())

['last_layer']


In [5]:
print(weights_dict_test['last_layer'][0])
print(weights_dict_test['last_layer'][1].shape)
print(weights_dict_test['last_layer'][1])

[[-0.00041662 -0.00353788  0.00106028 ... -0.00134582  0.00190614
  -0.00243913]
 [ 0.00408528  0.00418111 -0.00251761 ... -0.00158784 -0.0001661
   0.00020998]
 [-0.00277867 -0.0027039   0.00085511 ... -0.00456285  0.00487587
   0.00440324]
 ...
 [-0.00144065 -0.00295138 -0.00382247 ...  0.00146416  0.00394172
   0.00060865]
 [ 0.00153437 -0.00270551 -0.00297345 ...  0.00369073 -0.00077833
   0.00110156]
 [-0.0022927  -0.00305246  0.00286247 ...  0.00209831  0.00114476
   0.0038535 ]]
(12,)
[ 0.20129488  0.04828681  0.03133622  0.20117898  0.0758582  -0.13442625
  0.2165261  -0.04616061  0.11876933 -0.10707303  0.20810203  0.16723256]
